Tout d'abord, récupérer les fichiers de séquences génomiques fasta et les ranger dans un dossier "genomes".
Utiliser le fichier mainSURE.nf
Si besoin, modifier les chemins d'accès précisés dans le fichier (extrait ci-dessous) : 

In [ ]:
params.genomes_dir = '/path_to_/genomes'
params.concat_dir = '/path_to/Test_Nextflow'

#Modifier les chemins 

Lancer la pipeline nextflow :

In [ ]:
nextflow run mainSURE.nf

Le pipeline est pour l'instant composé uniquement d'un process qui exécute le fichier python extract_and_concatenate.py (voir ci-dessous)

In [ ]:
from Bio import SeqIO
import argparse
import re
from pathlib import Path
from collections import defaultdict

def extraction_chromosome(input_dir, output_dir):
    pattern = re.compile(r"(.+)\.genome\.fasta")
    chromosomes = defaultdict(list)

    # Parcourir tous les fichiers d'entrée
    for file in Path(input_dir).glob("*.genome.fasta"):
        for record in SeqIO.parse(file, "fasta"):
            search = re.search(r"Chromosome (C\d+)", record.description)
            if search:
                chromosomes[search.group(1)].append(record)

    # Créer le répertoire de sortie s'il n'existe pas
    output_dir.mkdir(parents=True, exist_ok=True)

    # Générer les fichiers concaténés
    for group, chr_list in chromosomes.items():
        output_file = output_dir / f"Concat_{group}.fasta"
        with open(output_file, "w") as concat:
            for chr in chr_list:
                concat.write(f">{chr.description}\n{chr.seq}\n")

    print("Fichiers FASTA générés.")

def main():
    # Configuration des arguments du script
    parser = argparse.ArgumentParser(description="Extraction et concaténation de chromosomes.")
    parser.add_argument("--input", required=True, help="/home/nguyeho3/Documents/Github_Pangenome/Projet_Pangenome/Test_Nextflow/genomes")
    parser.add_argument("--output", required=True, help="/home/nguyeho3/Documents/Github_Pangenome/Projet_Pangenome/Test_Nextflow/Concat2")
    args = parser.parse_args()

    # Appeler la fonction principale avec les arguments
    extraction_chromosome(Path(args.input), Path(args.output))

if __name__ == '__main__':
    main()


Utiliser la commande ci-dessous pour créer un index pour chaque fichier de concaténation.

In [ ]:
samtools faidx nom_du_fichier_concatene.fasta

Utiliser la commande ci-dessous dans le terminal pour utiliser pggb sur la séquence

In [ ]:
nohup pggb -i nom_du_fichier_concatene.fa -n 6 -o Results_PGGB --multiqc